In [1]:

import time
import random
import numpy as np
import math as m
from scipy import ndimage
import cv2
import itertools


from individual import Individual
from rotate import rotate

from __future__ import division

EPSILON = .0000001
total_agents = 400
TIMESTEP = 0.2
number_of_states = 10

def main():

    #Simulation start time
    t1 = time.clock()
    #Random generator engine from a time-based seed
    random.seed()
    
    #Set parameters

    arena_size = 1000
    top_left = np.array([0.0,0.0])
    bottom_right = np.array([arena_size,arena_size])

    
    sight_num = 166  #make even

    
    max_turning_rate = 114.5
    body_size = 4.0
    zoa = 12.0 * body_size
    zor = 2.0 * body_size
    zoo = 12.0 * body_size 
    speed = 10
    color = [255,135,71]
     
    angular_error_sd = 0.0
    
    ko = 0.25
    ka = 0.25
    
    
    agents = []
    
    
    #Simulation starts HERE
    
    #Set up agents
    for indx_agents in range(total_agents):
        set_direction = np.array([1.0, 0.0])    # need to be set to unit vectors
        set_direction = rotate(set_direction, random.random()*360.0)
        set_r_center = random_bounded_point(bottom_right, top_left)
        state = np.random.choice(np.arange(number_of_states))


        agents.append(Individual(set_r_center, set_direction, max_turning_rate,
                                 speed, zoa, zoo, zor, angular_error_sd, 
                                 body_size, state, ko, ka))
    
    for indx_j in itertools.count():
        
        move_agents(arena_size, zoa, agents)
        
        
        #draw to screen
        if indx_j % 1 == 0:
            kill = graphics(arena_size, agents, zoa)
            
        if kill:
            close_cv2()
            break
        
        
           

def close_cv2():
    for ind in range(10):
        cv2.destroyAllWindows()
        cv2.waitKey(1)

def move_agents(arena_size, zoa, agents):
    #reset each individuals z buffer, sight_vec, and zoa_count
    for i in range(len(agents)):
        agents[i].zoa_count = 0
        agents[i].zor_count = 0
        agents[i].total_zoa = np.zeros(2)
        agents[i].total_zoo = np.zeros(2)
        agents[i].total_zor = np.zeros(2)
        agents[i].state_count = np.zeros(number_of_states)

    linked_list(arena_size, zoa, agents)
    implement_social_forces(agents)

    
    for i in range(len(agents)):
        agents[i].move(arena_size, TIMESTEP)


def linked_list(arena_size, zoa, agents):

    # this implementation assumes equal x and y 
    # dimensions i.e. the arena is of size n x n

    
    # Part 1: List construction
    # this algorithm creates headers and linked lists (assigns a list of agents to a particular cell)
    # headers (head): holds index of first atom (or -1 if empty)
    # linked list (lscl): holds the agents index to which the ith agents points

    number_of_cells = int(arena_size // zoa)   # in a row/column
    cell_size = arena_size / number_of_cells      #width/height

    lscl = np.zeros(len(agents), np.int32)
    head = -1 * np.ones(number_of_cells * number_of_cells, np.int32) 
    cell_x = 0
    cell_y = 0

    for indx_agent in range(len(agents)):
        # c_x and c_y determine which cell agents is in
        cell_x = agents[indx_agent].r_center[0] // cell_size
        cell_y = agents[indx_agent].r_center[1] // cell_size
        
        #if cell_x < 0 or cell_x >= 10 or cell_y < 0 or cell_y >= 10: 
        #    print('cell_x, ',cell_x, ' cell_y ', cell_y)
        #    close_cv2()
        
        #calculate scalar cell id
        c = int(cell_x * number_of_cells + cell_y)  
        #link agents to previous occupant or -1 if first to be present there
        lscl[indx_agent] = head[c]                      
        #the last agents goes to the header
        head[c] = indx_agent                            
    
    # Part 2: Compute interactions
    for cell_x  in range(number_of_cells):
        for cell_y in range(number_of_cells):
           
            c = cell_x * number_of_cells + cell_y
            if head[c] != -1:

                for scan_neighbours_x in range(cell_x - 1, cell_x + 2):
                    for scan_neighbours_y in range(cell_y - 1, cell_y + 2):
                    
                        pbound_x = scan_neighbours_x;
                        pbound_y = scan_neighbours_y;
                        
                        if scan_neighbours_x < 0:
                            pbound_x += number_of_cells 
                        elif scan_neighbours_x >= number_of_cells:
                            pbound_x -= number_of_cells
                        
                        if scan_neighbours_y < 0:
                            pbound_y += number_of_cells
                        elif scan_neighbours_y >=number_of_cells:
                            pbound_y -= number_of_cells
                        
                        c1 = pbound_x * number_of_cells + pbound_y
                        
                        i = head[c]
                        
                        while i != -1:
                            j = head[c1]
                            
                            while j != -1:
                                if i != j:
                                    compute_social_forces(int(i), int(j), agents, arena_size)     
                                j = lscl[j]
                            i = lscl[i];


def compute_social_forces(i, j, agents, arena_size): 

    temp_vector = np.zeros(2);
    
    temp_vector = agents[j].r_center - agents[i].r_center
    
    if abs(temp_vector[0]) > arena_size / 2:

        if agents[i].r_center[0] < agents[j].r_center[0]:
            temp_vector[0] = (agents[j].r_center[0] 
                             - (agents[i].r_center[0] + arena_size))
        else:
            temp_vector[0] = (agents[j].r_center[0]
                             - (agents[i].r_center[0] - arena_size))

    if abs(temp_vector[1]) > arena_size / 2:

        if (agents[i].r_center[1] < agents[j].r_center[1]):
            temp_vector[1] = (agents[j].r_center[1] 
                             - (agents[i].r_center[1] + arena_size))
        else:
            temp_vector[1] = (agents[j].r_center[1] 
                             - (agents[i].r_center[1] - arena_size))

    dist = np.linalg.norm(temp_vector)
    dist = np.maximum(dist, EPSILON)
    
    temp_vector_norm = temp_vector / dist
    
    if dist < agents[i].zone_of_repulsion:
        agents[i].total_zor += (-temp_vector_norm)
        agents[i].zor_count += 1
        agents[i].state_count[agents[j].state] += 1
        
        
    elif dist < agents[i].zone_of_attraction: 
        if dist < agents[i].zone_of_orientation:
            agents[i].total_zoo += agents[j].direction
        agents[i].total_zoa += temp_vector
        agents[i].zoa_count += 1
        agents[i].state_count[agents[j].state] += 1
        

def normalize(vec):
    vec_mag = np.linalg.norm(vec)
    vec_mag = np.maximum(vec_mag, EPSILON)
    vec_norm = vec / vec_mag
    return vec_norm
            



def implement_social_forces(agents):

    #now have total_zod, total_zoo and total_zoa calculated for all individuals
    for agent_indx in range(total_agents):
        if agents[agent_indx].zor_count > 0:
            if (np.absolute(agents[agent_indx].total_zor[0]) < EPSILON and 
                np.absolute(agents[agent_indx].total_zor[1]) < EPSILON):
                    agents[agent_indx].desired_direction = agents[agent_indx].direction 
            else:
                agents[agent_indx].desired_direction = normalize(agents[agent_indx].total_zor)


        elif agents[agent_indx].zoa_count > 0:
            agents[agent_indx].desired_direction = (
                normalize(agents[agent_indx].total_zoo) * agents[agent_indx].ko +
                normalize(agents[agent_indx].total_zoa) * agents[agent_indx].ka + 
                agents[agent_indx].direction * (1 - agents[agent_indx].ko - agents[agent_indx].ka))
            
            if (np.absolute(agents[agent_indx].desired_direction[0]) < EPSILON and 
                np.absolute(agents[agent_indx].desired_direction[1]) < EPSILON):
                
                agents[agent_indx].desired_direction = agents[agent_indx].direction
                
            else:
                
                agents[agent_indx].desired_direction = normalize(agents[agent_indx].desired_direction)
        
        if agents[agent_indx].zor_count > 0 or agents[agent_indx].zoa_count > 0:
            agents[agent_indx].state = np.argmax(agents[agent_indx].state_count)



def random_bounded_point(bottom_right, top_left):

    # Create randomly distributed co-ordinate in the simulated world
    range_x = bottom_right[0] - top_left[0]
    range_y = bottom_right[1] - top_left[1]
    
    random_x = random.random()
    random_y = random.random()
    
    random_x *= range_x
    random_y *= range_y
    random_point = np.array([random_x, random_y])
    
    return random_point



def graphics(arena_size, agents, zoa):
    
    body_scale = .75

    number_of_cells = int(arena_size // zoa)   # in a row/column
    cell_size = arena_size / number_of_cells      #width/height

    
    colors = [[244,66,66],[244,158,66],[244,232,66],[185,244,66],[125,244,66],
              [66,244,200],[66,217,244],[66,101,244],[176,66,244],[244,66,170],
              [255,135,71],[255,255,255]]

    # Draw arena
    visualisation = np.zeros((arena_size, arena_size, 3), np.uint8)
    
    # Draw agents


    for i  in range(len(agents)):
        

        color = colors[agents[i].state]

        
        cv2.circle(visualisation, (int(agents[i].r_center[0]), 
            int(agents[i].r_center[1])), int(agents[i].body_size * body_scale), 
            color, -1, cv2.LINE_AA)


        cv2.line(visualisation, (int(agents[i].r_center[0]), 
            int(agents[i].r_center[1])), (int(agents[i].r_center[0] - agents[i].direction[0] * 8.0),
                                          int(agents[i].r_center[1] - agents[i].direction[1] * 8.0)), color, 1, cv2.LINE_AA)
                       
    
    cv2.imshow("decision_making", visualisation)
    return (cv2.waitKey(1) & 0xFF == 27)


if __name__ == "__main__":
    main()

In [3]:
import numpy as np
import math as m
vec = np.ones(10)
vec = np.linalg.norm(vec)
print(vec)

3.16227766017


In [7]:
np.sqrt(np.sum(np.ones(10)**2))**2

10.000000000000002

In [8]:
3.3*3.3

10.889999999999999